In [20]:
import re
from datasketch import MinHash, MinHashLSH

def preprocess_text(text):
    """ Entfernt Sonderzeichen und führt eine einfache Textbereinigung durch. """
    text = re.sub(r'[^\w\s]', '', text).lower()
    return text

def create_shingles(text, k=2):
    """ Erstellt Shingles (n-Gramme) der Länge k aus dem Text. """
    words = text.split()
    for i in range(len(words) - k + 1):
        yield ' '.join(words[i:i + k])

# Lese die Datei und erstelle MinHash für jeden Artikel
file_path = '/Users/furkan/Downloads/articles-2.txt'
lsh = MinHashLSH(threshold=0.5, num_perm=328)  # Threshold und Anzahl der Permutationen
articles_minhash = {}

with open(file_path, 'r', encoding='utf-8') as file:
    content = file.readlines()

for line in content:
    parts = line.split(' ', 1)
    if len(parts) == 2:
        article_id, article_text = parts[0], parts[1]
        processed_text = preprocess_text(article_text)
        m = MinHash(num_perm=328)
        for shingle in create_shingles(processed_text):
            m.update(shingle.encode('utf-8'))
        lsh.insert(article_id, m)
        articles_minhash[article_id] = m

# Suche nach ähnlichen Artikel-Paaren und berechne die Jaccard-Ähnlichkeit
plagiarism_candidates = []
for article_id in articles_minhash:
    result = lsh.query(articles_minhash[article_id])
    for similar_id in result:
        if similar_id != article_id:
            similarity = articles_minhash[article_id].jaccard(articles_minhash[similar_id])
            plagiarism_candidates.append((article_id, similar_id, similarity))

# Ergebnisse ausgeben
for pair in plagiarism_candidates:
    print(f"Artikel {pair[0]} und {pair[1]} könnten ähnlich sein mit einer Ähnlichkeit von {pair[2]*100:.2f}%.")


Artikel t980 und t2023 könnten ähnlich sein mit einer Ähnlichkeit von 97.66%.
Artikel t1088 und t5015 könnten ähnlich sein mit einer Ähnlichkeit von 97.66%.
Artikel t1297 und t4638 könnten ähnlich sein mit einer Ähnlichkeit von 98.44%.
Artikel t1768 und t5248 könnten ähnlich sein mit einer Ähnlichkeit von 96.09%.
Artikel t1952 und t3495 könnten ähnlich sein mit einer Ähnlichkeit von 98.44%.
Artikel t2023 und t980 könnten ähnlich sein mit einer Ähnlichkeit von 97.66%.
Artikel t2535 und t8642 könnten ähnlich sein mit einer Ähnlichkeit von 97.66%.
Artikel t2839 und t9303 könnten ähnlich sein mit einer Ähnlichkeit von 99.22%.
Artikel t2957 und t7111 könnten ähnlich sein mit einer Ähnlichkeit von 99.22%.
Artikel t3268 und t7998 könnten ähnlich sein mit einer Ähnlichkeit von 98.44%.
Artikel t3466 und t7563 könnten ähnlich sein mit einer Ähnlichkeit von 96.88%.
Artikel t3495 und t1952 könnten ähnlich sein mit einer Ähnlichkeit von 98.44%.
Artikel t4638 und t1297 könnten ähnlich sein mit einer